Never_Summer_100k.gpx

In [1]:
import gpxpy
import gpxpy.gpx
import folium

# Load and parse the GPX file
with open('Never_Summer_100k.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Get the coordinates of the track
coordinates = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            coordinates.append((point.latitude, point.longitude))

# Find the starting point and create the map
start_point = coordinates[0]
m = folium.Map(location=start_point, zoom_start=14)

# Add the GPX track to the map
folium.PolyLine(coordinates, color='blue', weight=5).add_to(m)

# Display the map
m


In [30]:
import gpxpy
import random
import numpy as np
from scipy.interpolate import interp1d
from datetime import timedelta
from scipy.spatial import distance
import datetime

def generate_tracking_points(gpx, probability, fatigue_function, noise_factor=0.0004):

    # Extract coordinates and time from the GPX file
    coordinates = []
    elevations = []
    times = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append((point.latitude, point.longitude))
                elevations.append(point.elevation)
                times.append(point.time.timestamp())

    # Compute total distance and cumulative distances
    distances = [0]
    for i in range(1, len(coordinates)):
        d = distance.euclidean(coordinates[i - 1], coordinates[i])
        distances.append(distances[-1] + d)

    # Initialize tracking points
    tracking_points = []

    # Calculate the tracking start time
    start_time = times[0]
    tracking_start_time = start_time - random.randint(1, 10) * 60
    
    print(f"Starting at {tracking_start_time} based on gpx starting at {start_time}")

    # Generate tracking points
    current_time = tracking_start_time
    print("Starting")
    while current_time < times[0]:
        current_time += timedelta(minutes=10).total_seconds()
    while current_time <= times[-1]:
        # Check if a tracking point should be generated
        if random.random() < probability:
            # Interpolate the position and elevation based on the current time
            lat_interp = interp1d(times, [coord[0] for coord in coordinates])
            lon_interp = interp1d(times, [coord[1] for coord in coordinates])
            ele_interp = interp1d(times, elevations)

            lat = lat_interp(current_time)
            lon = lon_interp(current_time)
            ele = ele_interp(current_time)

            # Add noise to the position
            lat += np.random.normal(0, noise_factor)
            lon += np.random.normal(0, noise_factor)

            # Create a tracking point with the interpolated position and elevation
            tracking_point = gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele, time=datetime.datetime.fromtimestamp(current_time))
            tracking_points.append(tracking_point)

        # Update the current time based on the fatigue function
        current_time += timedelta(minutes=10).total_seconds() * fatigue_function(current_time, start_time, times)

    return tracking_points

# Example usage:
probability = 0.95

def fatigue_function(current_time, start_time, times):
    # Example fatigue function: slowdown at the beginning and speed up near the end
    # Customize this function based on your requirements
    time_fraction = (current_time - start_time) / (times[-1] - start_time)
    if time_fraction < 0.8:
        return 1.2
    else:
        return 0.8

generated_points = generate_tracking_points(gpx, probability, fatigue_function)



Starting at 1500723045.0 based on gpx starting at 1500723105.0
Starting


In [32]:
def plot_gpx_and_generated_points(coordinates, generated_points):
    # Find the starting point and create the map
    start_point = coordinates[0]
    m = folium.Map(location=start_point, zoom_start=14)

    # Add the GPX track to the map
    folium.PolyLine(coordinates, color='blue', weight=5).add_to(m)

    # Add the generated points as red dots
    for point in generated_points:
        folium.CircleMarker(
            location=(point.latitude, point.longitude),
            radius=5,
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(m)

    # Display the map
    return m

map_with_points = plot_gpx_and_generated_points(coordinates, generated_points)
map_with_points


In [59]:
import folium
import gpxpy
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, fixed

def load_gpx_data(gpx_file):
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)

    coordinates = []
    elevations = []
    times = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append((point.latitude, point.longitude))
                elevations.append(point.elevation)
                times.append(point.time)
    return coordinates, elevations, times

def create_map(coordinates):
    start_point = coordinates[0]
    m = folium.Map(location=start_point, zoom_start=14)
    folium.PolyLine(coordinates, color='blue', weight=5).add_to(m)
    return m

def plot_elevation_profile(elevations, times):
    plt.figure(figsize=(10, 4))
    plt.plot(times, elevations)
    plt.xlabel("Time")
    plt.ylabel("Elevation (m)")
    plt.grid()

def update_plot(slider_value, gpx_file, generated_points):
    race_time = generated_points[slider_value].time
    coordinates, elevations, times = load_gpx_data(gpx_file)
    m = create_map(coordinates)
    
    for point in generated_points[:slider_value]:
        folium.CircleMarker(
            location=(point.latitude, point.longitude),
            radius=5,
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(m)
    
    display(m)
    #plot_elevation_profile(elevations, times)
    #plt.scatter([race_time] * slider_value, elevations[:slider_value], color='red')
    #plt.show()



In [60]:
gpx_file = 'Never_Summer_100k.gpx'
slider = IntSlider(min=0, max=len(generated_points) - 1, step=1, value=0)
interact(update_plot, slider_value=slider, gpx_file=fixed(gpx_file), generated_points=fixed(generated_points))

interactive(children=(IntSlider(value=0, description='slider_value', max=92), Output()), _dom_classes=('widget…

<function __main__.update_plot(slider_value, gpx_file, generated_points)>

In [58]:
generated_points[0]

GPXTrackPoint(40.50497682611275, -105.99445417910039, elevation=array(2797.), time=datetime.datetime(2017, 7, 22, 5, 40, 45))